In [ ]:
import numpy
import spacy
import pickle

In [ ]:
import shutil
from requests import get
from google.colab import drive,files
drive.mount("/content/drive")
import pickle
with open('./data/restrain.pickle','rb') as file:
    localtrain = pickle.load(file)
with open('./data/restest.pickle','rb') as file:
    localtest = pickle.load(file)

Mounted at /content/drive


In [ ]:
!pip install -q transformers

     |████████████████████████████████| 4.2 MB 8.3 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 52.4 MB/s 
     |████████████████████████████████| 596 kB 66.9 MB/s 


In [ ]:
import json
import argparse
from collections import Counter
import re
import pickle
from tqdm import tqdm
from transformers import BertTokenizer
import os
import sys
import copy
import random
import logging
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import numpy as np
from sklearn import metrics
from time import strftime, localtime
from transformers import BertModel,AdamW

In [ ]:
def ParseData(data_path):
    with open(data_path) as infile:
        all_data = []
        data = json.load(infile)
        for d in data:
            for aspect in d['aspects']:
                text_list = list(d['token'])
                tok = list(d['token'])       # word token
                length = len(tok)            # real length
                # if args.lower == True:
                tok = [t.lower() for t in tok]
                tok = ' '.join(tok)
                asp = list(aspect['term'])   # aspect
                asp = [a.lower() for a in asp]
                asp = ' '.join(asp)
                label = aspect['polarity']   # label
                pos = list(d['pos'])         # pos_tag 
                head = list(d['head'])       # head
                deprel = list(d['deprel'])   # deprel
                # position
                aspect_post = [aspect['from'], aspect['to']] 
                post = [i-aspect['from'] for i in range(aspect['from'])] \
                       +[0 for _ in range(aspect['from'], aspect['to'])] \
                       +[i-aspect['to']+1 for i in range(aspect['to'], length)]
                # aspect mask
                if len(asp) == 0:
                    mask = [1 for _ in range(length)]    # for rest16
                else:
                    mask = [0 for _ in range(aspect['from'])] \
                       +[1 for _ in range(aspect['from'], aspect['to'])] \
                       +[0 for _ in range(aspect['to'], length)]
                
                sample = {'text': tok, 'aspect': asp, 'pos': pos, 'post': post, 'head': head,\
                          'deprel': deprel, 'length': length, 'label': label, 'mask': mask, \
                          'aspect_post': aspect_post, 'text_list': text_list}
                all_data.append(sample)

    return all_data

class Vocab(object):
    ''' vocabulary of dataset '''
    def __init__(self, vocab_list, add_pad, add_unk):
        self._vocab_dict = dict()
        self._reverse_vocab_dict = dict()
        self._length = 0
        if add_pad:
            self.pad_word = '<pad>'
            self.pad_id = self._length
            self._length += 1
            self._vocab_dict[self.pad_word] = self.pad_id
        if add_unk:
            self.unk_word = '<unk>'
            self.unk_id = self._length
            self._length += 1
            self._vocab_dict[self.unk_word] = self.unk_id
        for w in vocab_list:
            self._vocab_dict[w] = self._length
            self._length += 1
        for w, i in self._vocab_dict.items():   
            self._reverse_vocab_dict[i] = w  
    
    def word_to_id(self, word):  
        if hasattr(self, 'unk_id'):
            return self._vocab_dict.get(word, self.unk_id)
        return self._vocab_dict[word]
    
    def id_to_word(self, id_):   
        if hasattr(self, 'unk_word'):
            return self._reverse_vocab_dict.get(id_, self.unk_word)
        return self._reverse_vocab_dict[id_]
    
    def has_word(self, word):
        return word in self._vocab_dict
    
    def __len__(self):
        return self._length
    
    @staticmethod
    def load_vocab(vocab_path: str):
        with open(vocab_path, "rb") as f:
            return pickle.load(f)

    def save_vocab(self, vocab_path):
        with open(vocab_path, "wb") as f:
            pickle.dump(self, f)

def softmax(x):
    if len(x.shape) > 1:
        # matrix
        tmp = np.max(x, axis=1)
        x -= tmp.reshape((x.shape[0], 1))
        x = np.exp(x)
        tmp = np.sum(x, axis=1)
        x /= tmp.reshape((x.shape[0], 1))
    else:
        # vector
        tmp = np.max(x)
        x -= tmp
        x = np.exp(x)
        tmp = np.sum(x)
        x /= tmp
    return x

In [ ]:
class Tokenizer4BertGCN:
    def __init__(self, max_seq_len, pretrained_bert_name):
        self.max_seq_len = max_seq_len
        self.tokenizer = BertTokenizer.from_pretrained(pretrained_bert_name)
        self.cls_token_id = self.tokenizer.cls_token_id
        self.sep_token_id = self.tokenizer.sep_token_id
    def tokenize(self, s):
        return self.tokenizer.tokenize(s)
    def convert_tokens_to_ids(self, tokens):
        return self.tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
fnametrainres = './dataset/Restaurants_corenlp/train.json'
fnametestres = './dataset/Restaurants_corenlp/test.json'
tokenizer = Tokenizer4BertGCN(85, 'bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class globaltrainABSAGCNDATA(Dataset):
  def __init__(self,fname,tokenizer):
    self.data = []
    parse = ParseData
    polarity_dict = {'positive':0,'negative':1,'neutral':2}
    for obj in tqdm(parse(fname), total=len(parse(fname)), desc="Training examples"):
      polarity = polarity_dict[obj['label']]
      text = obj['text']
      term = obj['aspect']
      term_start = obj['aspect_post'][0]
      term_end = obj['aspect_post'][1]
      text_list = obj['text_list']
      left, term, right = text_list[: term_start], text_list[term_start: term_end], text_list[term_end: ]

      #polarity score
      senticNet = {}
      with open('./senticnet/senticnetword.txt','r') as file:
        fp = file
        for line in fp:
          line = line.strip()
          if not line:
            continue
          word,sentic = line.split('\t')
          senticNet[word] = sentic
        fp.close()
      senticvalue = np.zeros(context_len)
      for word_i,bword in enumerate(text_list):
        if str(bword) in senticNet:
          sentic = 1+abs(float(senticNet[str(bword)]))#abs(float(senticNet[str(btoken)]))
        else:
          sentic = 1
        senticvalue[word_i] = sentic
      senticvalue = torch.from_numpy(senticvalue)

      #SRD
      asp_avg_index = (len(left)*2+len(term)-1)/2
      context_len = len(text_list)
      for i in range(context_len):
        srd = abs(i-asp_avg_index)-len(term)/2
        if srd > 3:
          senticvalue[i] = senticvalue[i]
        else:
          senticvalue[i] = senticvalue[i]+1
      
      
      left_tokens, term_tokens, right_tokens = [], [], []
      left_tok2ori_map, term_tok2ori_map, right_tok2ori_map = [], [], []
      left_sentic, term_sentic, right_sentic = [], [], [] 
      for ori_i, w in enumerate(left):
          for t in tokenizer.tokenize(w):
              left_tokens.append(t)                   # * ['expand', '##able', 'highly', 'like', '##ing']
              left_tok2ori_map.append(ori_i)          # * [0, 0, 1, 2, 2]
              left_sentic.append(senticvalue[ori_i])
      asp_start = len(left_tokens)  
      offset = len(left) 
      for ori_i, w in enumerate(term):        
          for t in tokenizer.tokenize(w):
              term_tokens.append(t)
              term_tok2ori_map.append(ori_i + offset)
              term_sentic.append(senticvalue[ori_i+ offset])
      asp_end = asp_start + len(term_tokens)
      offset += len(term) 
      for ori_i, w in enumerate(right):
          for t in tokenizer.tokenize(w):
              right_tokens.append(t)
              right_tok2ori_map.append(ori_i+offset)
              term_sentic.append(senticvalue[ori_i+ offset])
      while len(left_tokens) + len(right_tokens) > tokenizer.max_seq_len-2*len(term_tokens) - 3:
          if len(left_tokens) > len(right_tokens):
              left_tokens.pop(0)
              left_tok2ori_map.pop(0)
          else:
              right_tokens.pop()
              right_tok2ori_map.pop()
              
      bert_tokens = left_tokens + term_tokens + right_tokens
      tok2ori_map = left_tok2ori_map + term_tok2ori_map + right_tok2ori_map
      sentic_token = left_sentic + term_sentic + right_sentic
      truncate_tok_len = len(bert_tokens)

         
      # context raw 
      context_raw_asp_ids = [tokenizer.cls_token_id]+tokenizer.convert_tokens_to_ids(bert_tokens)+[tokenizer.sep_token_id]
      context_raw_asp_len = len(context_raw_asp_ids)
      paddings_raw = [0] * (tokenizer.max_seq_len - context_raw_asp_len)
      context_raw_asp_ids += paddings_raw
      context_raw_asp_ids = np.asarray(context_raw_asp_ids,dtype='int64')
      #context+aspect
      context_asp_ids = [tokenizer.cls_token_id]+tokenizer.convert_tokens_to_ids(
          bert_tokens)+[tokenizer.sep_token_id]+tokenizer.convert_tokens_to_ids(term_tokens)+[tokenizer.sep_token_id]
      context_asp_len = len(context_asp_ids)
      paddings = [0] * (tokenizer.max_seq_len - context_asp_len)
      context_len = len(bert_tokens)
      context_asp_seg_ids = [0] * (1 + context_len + 1) + [1] * (len(term_tokens) + 1) + paddings
      #src_mask = [0] + [1] * context_len + [0] * (opt.max_length - context_len - 1)
      src_mask = [0] + [1]*context_len + [0] * (85 - context_len - 1)
      aspect_mask = [0] + [0] * asp_start + [1] * (asp_end - asp_start)
      aspect_mask = aspect_mask + (85 - len(aspect_mask)) * [0]
      context_asp_attention_mask = [1] * context_asp_len + paddings
      context_asp_ids += paddings
      #aspect bert 
      aspect_bert_indices = [tokenizer.cls_token_id]+tokenizer.convert_tokens_to_ids(term_tokens) + [tokenizer.sep_token_id]
      aspect_bert_indices_len = len(aspect_bert_indices)
      paddings_abi = [0]*(tokenizer.max_seq_len-aspect_bert_indices_len)
      aspect_bert_indices += paddings_abi
      aspect_bert_indices = np.asarray(aspect_bert_indices,dtype='int64')
      #to numpy
      context_asp_ids = np.asarray(context_asp_ids, dtype='int64')
      context_asp_seg_ids = np.asarray(context_asp_seg_ids, dtype='int64')
      context_asp_attention_mask = np.asarray(context_asp_attention_mask, dtype='int64')
      src_mask = np.asarray(src_mask, dtype='int64')
      aspect_mask = np.asarray(aspect_mask,dtype='int64')

      texts = context_raw_bert_indices.cpu().numpy()
      asps = aspect_bert_indices.cpu().numpy()
  
      for text_i,asp_i in zip(range(len(texts)),range(len(asps))):
        asp_len = np.count_nonzero(asps[asp_i]) - 2
        try:
          asp_begin = np.argwhere(texts[text_i]==asps[asp_i][1])[0][0]
          asp_avg_index = (asp_begin*2+asp_len-1)/2
        except:
          continue
        for i in range(1,np.count_nonzero(texts[text_i])-1):
          srd = abs(i-asp_avg_index) + (asp_len-1)/2
          if srd>3:
            adj[text_i,85] = adj[text_i,85]
          else:
            adj[text_i,85] = 1+adj[text_i,85]
      #srd
      texts = text_raw_bert_indices.cpu().numpy()
      asps = aspect_bert_indices.cpu().numpy()
      term_len = term_start - term_end
      text_len = len(text_list)
      term_avg_index = (term_start*2+term_len-1)/2
      for i in range(0,np.count_nonzero(texts[text_i])):
        srd = abs(tok2ori_map[i]-term_avg_index) + (term_len-1)/2
        if srd > 3:
          sentic_token[i] = sentic_token[i]
        else:
          sentic_token[i] = sentic_token[i]+1
      
      # pad adj
      sentictrain = []
      for i in range(len(localtrain)):
        a = localtrain[i]['adj_matrix']
        a = torch.from_numpy(a)
        b = torch.zeros(85,1)
        c = torch.zeros(1,86)
        d = torch.cat((a,b),1)
        e = torch.cat((d,c),0)
        e[1:context_len+1,85] = sentic_token
        e[85,1:context_len+1] = sentic_token
        senticdata = {
            'adj_matrix':e,
        }
        yuantrain.append(yuandata)
      data = {
          'text_bert_indices': context_asp_ids,
          'text_raw_bert_indices':context_raw_asp_ids,
          'aspect_bert_indices': aspect_bert_indices,
          'bert_segments_ids': context_asp_seg_ids,
          'attention_mask': context_asp_attention_mask,
          'asp_start': asp_start,
          'asp_end': asp_end,
          'src_mask': src_mask,
          'aspect_mask': aspect_mask,
          'polarity': polarity,
      }
      self.data.append(data)
      #self.data.update(localtrain)
    for i in range(0,len(self.data)):
      self.data[i].update(yuantrain[i])

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

In [ ]:
class globaltestABSAGCNDATA(Dataset):
  def __init__(self,fname,tokenizer):
    self.data = []
    parse = ParseData
    polarity_dict = {'positive':0,'negative':1,'neutral':2}
    for obj in tqdm(parse(fname), total=len(parse(fname)), desc="Training examples"):
      polarity = polarity_dict[obj['label']]
      text = obj['text']
      term = obj['aspect']
      term_start = obj['aspect_post'][0]
      term_end = obj['aspect_post'][1]
      text_list = obj['text_list']
      left, term, right = text_list[: term_start], text_list[term_start: term_end], text_list[term_end: ]

      #polarity score
      #polarity score
      senticNet = {}
      with open('./senticnet/senticnetword.txt','r') as file:
        fp = file
        for line in fp:
          line = line.strip()
          if not line:
            continue
          word,sentic = line.split('\t')
          senticNet[word] = sentic
        fp.close()
      senticvalue = np.zeros(context_len)
      for word_i,bword in enumerate(text_list):
        if str(bword) in senticNet:
          sentic = 1+abs(float(senticNet[str(bword)]))#abs(float(senticNet[str(btoken)]))
        else:
          sentic = 1
        senticvalue[word_i] = sentic
      senticvalue = torch.from_numpy(senticvalue)

      #SRD
      asp_avg_index = (len(left)*2+len(term)-1)/2
      context_len = len(text_list)
      for i in range(context_len):
        srd = abs(i-asp_avg_index)-len(term)/2
        if srd > 3:
          senticvalue[i] = senticvalue[i]
        else:
          senticvalue[i] = senticvalue[i]+1


      left_tokens, term_tokens, right_tokens = [], [], []
      left_tok2ori_map, term_tok2ori_map, right_tok2ori_map = [], [], []
      left_sentic, term_sentic, right_sentic = [], [], [] 
      for ori_i, w in enumerate(left):
          for t in tokenizer.tokenize(w):
              left_tokens.append(t)                   # * ['expand', '##able', 'highly', 'like', '##ing']
              left_tok2ori_map.append(ori_i)          # * [0, 0, 1, 2, 2]
              left_sentic.append(senticvalue[ori_i])
      asp_start = len(left_tokens)  
      offset = len(left) 
      for ori_i, w in enumerate(term):        
          for t in tokenizer.tokenize(w):
              term_tokens.append(t)
              term_tok2ori_map.append(ori_i + offset)
              term_sentic.append(senticvalue[ori_i+ offset])
      asp_end = asp_start + len(term_tokens)
      offset += len(term) 
      for ori_i, w in enumerate(right):
          for t in tokenizer.tokenize(w):
              right_tokens.append(t)
              right_tok2ori_map.append(ori_i+offset)
              term_sentic.append(senticvalue[ori_i+ offset])
      while len(left_tokens) + len(right_tokens) > tokenizer.max_seq_len-2*len(term_tokens) - 3:
          if len(left_tokens) > len(right_tokens):
              left_tokens.pop(0)
              left_tok2ori_map.pop(0)
          else:
              right_tokens.pop()
              right_tok2ori_map.pop()
              
      bert_tokens = left_tokens + term_tokens + right_tokens
      tok2ori_map = left_tok2ori_map + term_tok2ori_map + right_tok2ori_map
      sentic_token = left_sentic+term_sentic+right_sentic
      truncate_tok_len = len(bert_tokens)
         
      # context raw 
      context_raw_asp_ids = [tokenizer.cls_token_id]+tokenizer.convert_tokens_to_ids(bert_tokens)+[tokenizer.sep_token_id]
      context_raw_asp_len = len(context_raw_asp_ids)
      paddings_raw = [0] * (tokenizer.max_seq_len - context_raw_asp_len)
      context_raw_asp_ids += paddings_raw
      context_raw_asp_ids = np.asarray(context_raw_asp_ids,dtype='int64')
      #context+aspect
      context_asp_ids = [tokenizer.cls_token_id]+tokenizer.convert_tokens_to_ids(
          bert_tokens)+[tokenizer.sep_token_id]+tokenizer.convert_tokens_to_ids(term_tokens)+[tokenizer.sep_token_id]
      context_asp_len = len(context_asp_ids)
      paddings = [0] * (tokenizer.max_seq_len - context_asp_len)
      context_len = len(bert_tokens)
      context_asp_seg_ids = [0] * (1 + context_len + 1) + [1] * (len(term_tokens) + 1) + paddings
      #src_mask = [0] + [1] * context_len + [0] * (opt.max_length - context_len - 1)
      src_mask = [0] + [1]*context_len + [0] * (85 - context_len - 1)
      aspect_mask = [0] + [0] * asp_start + [1] * (asp_end - asp_start)
      aspect_mask = aspect_mask + (85 - len(aspect_mask)) * [0]
      context_asp_attention_mask = [1] * context_asp_len + paddings
      context_asp_ids += paddings
      #aspect bert 
      aspect_bert_indices = [tokenizer.cls_token_id]+tokenizer.convert_tokens_to_ids(term_tokens) + [tokenizer.sep_token_id]
      aspect_bert_indices_len = len(aspect_bert_indices)
      paddings_abi = [0]*(tokenizer.max_seq_len-aspect_bert_indices_len)
      aspect_bert_indices += paddings_abi
      aspect_bert_indices = np.asarray(aspect_bert_indices,dtype='int64')
      #to numpy
      context_asp_ids = np.asarray(context_asp_ids, dtype='int64')
      context_asp_seg_ids = np.asarray(context_asp_seg_ids, dtype='int64')
      context_asp_attention_mask = np.asarray(context_asp_attention_mask, dtype='int64')
      src_mask = np.asarray(src_mask, dtype='int64')
      aspect_mask = np.asarray(aspect_mask,dtype='int64')
      
      # pad adj
      sentictest = []
      for i in range(len(localtest)):
        a = localtest[i]['adj_matrix']
        a = torch.from_numpy(a)
        b = torch.zeros(85,1)
        c = torch.zeros(1,86)
        d = torch.cat((a,b),1)
        e = torch.cat((d,c),0)
        e[1:context_len+1,85] = senticvalue
        e[85,1:context_len+1] = senticvalue
        senticdata = {
            'adj_matrix':e,
        }
        yuantrain.append(yuandata)
      data = {
          'text_bert_indices': context_asp_ids,
          'text_raw_bert_indices':context_raw_asp_ids,
          'aspect_bert_indices': aspect_bert_indices,
          'bert_segments_ids': context_asp_seg_ids,
          'attention_mask': context_asp_attention_mask,
          'asp_start': asp_start,
          'asp_end': asp_end,
          'src_mask': src_mask,
          'aspect_mask': aspect_mask,
          'polarity': polarity,
      }
      self.data.append(data)
      #self.data.update(localtrain)
    for i in range(0,len(self.data)):
      self.data[i].update(sentictest[i])

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

model

In [ ]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [ ]:
class globalGCNBert(nn.Module):
  def __init__(self,bert,opt,num_layers):
    super(globalGCNBert,self).__init__()
    self.bert = bert
    self.opt = opt
    self.layers = num_layers
    self.mem_dim = opt.bert_dim // 2
    self.bert_dim = opt.bert_dim
    self.bert_drop = nn.Dropout(opt.bert_dropout) #0.3
    self.pooled_drop = nn.Dropout(opt.bert_dropout) #0.3
    self.gcn_drop = nn.Dropout(opt.gcn_dropout) #0.1
    self.layernorm = LayerNorm(opt.bert_dim)

    self.W = nn.ModuleList()
    for layer in range(self.layers):
      input_dim = self.bert_dim if layer == 0 else self.mem_dim
      self.W.append(nn.Linear(input_dim, self.mem_dim))

  def forward(self,adj,inputs):
    text_bert_indices,text_raw_bert_indices,aspect_bert_indices,bert_segments_ids, attention_mask, asp_start, asp_end, adj_dep, src_mask, aspect_mask = inputs
    src_mask = src_mask.unsqueeze(-2)
      
    sequence_output = self.bert(text_bert_indices, attention_mask=attention_mask,token_type_ids=bert_segments_ids)[0]
    super_node = torch.mean(sequence_output,dim=1).unsqueeze(1)
    super_node = torch.zeros_like(super_node)
    pooled_output = self.bert(text_bert_indices, attention_mask=attention_mask,token_type_ids=bert_segments_ids)[1]
    gcn_inputs = self.layernorm(sequence_output)
    gcn_inputs = self.bert_drop(gcn_inputs)
    pooled_output = self.pooled_drop(pooled_output)
    gcn_inputs = torch.cat((gcn_inputs,super_node),1)
    
    texts = text_raw_bert_indices.cpu().numpy()
    asps = aspect_bert_indices.cpu().numpy()

    for text_i,asp_i in zip(range(len(texts)),range(len(asps))):
      asp_len = np.count_nonzero(asps[asp_i]) - 2
      try:
        asp_begin = np.argwhere(texts[text_i]==asps[asp_i][1])[0][0]
        asp_avg_index = (asp_begin*2+asp_len-1)/2
      except:
        continue
      for i in range(1,np.count_nonzero(texts[text_i])-1):
        srd = abs(i-asp_avg_index) + (asp_len-1)/2
        if srd>3:
          adj[text_i,85] = adj[text_i,85]
        else:
          adj[text_i,85] = 1+adj[text_i,85]
         
    denom_dep = adj.sum(2).unsqueeze(2) + 1  
    outputs_dep = gcn_inputs

    for l in range(self.layers):
      # ************SynGCN*************
      Ax_dep = adj.bmm(outputs_dep)
      AxW_dep = self.W[l](Ax_dep)
      AxW_dep = AxW_dep / denom_dep
      #AxW_dep = torch.mul(AxW_dep,memweight_text_raw_indices_1) if l>1 else AxW_dep
      gAxW_dep = F.relu(AxW_dep)
      outputs_dep = self.gcn_drop(gAxW_dep) if l < self.layers - 1 else gAxW_dep
    return outputs_dep,pooled_output

In [ ]:
class globalGCNAbsaModel(nn.Module):
  def __init__(self,bert,opt):
    super().__init__()
    self.opt = opt
    self.gcn = globalGCNBert(bert,opt,opt.num_layers)
  def forward(self,inputs):
    text_bert_indices,text_raw_bert_indices,aspect_bert_indices,bert_segments_ids, attention_mask, asp_start, asp_end, adj_dep, src_mask, aspect_mask = inputs
    h,pooled_output = self.gcn(adj_dep,inputs)
    h = h[:,:85,:].to('cuda')

    # avg pooling asp feature
    asp_wn = aspect_mask.sum(dim=1).unsqueeze(-1)
    aspect_mask = aspect_mask.unsqueeze(-1).repeat(1, 1, self.opt.bert_dim // 2) 
    outputs = (h*aspect_mask).sum(dim=1)/asp_wn
    #return outputs,adj_dep
    return outputs,adj_dep,pooled_output

In [ ]:
class globalGCNBertClassifier(nn.Module):
  def __init__(self,bert,opt):
    super().__init__()
    self.opt = opt
    self.gcn_model = globalGCNAbsaModel(bert,opt=opt)
    self.classifier = nn.Linear(opt.hidden_dim,opt.polarities_dim)
  def forward(self,inputs):
    outputs,adj_dep,pooled_output = self.gcn_model(inputs)
    logits = self.classifier(outputs)
    return logits,None

train

In [ ]:
import os
import sys
import copy
import random
import logging
import argparse
import torch
import torch.nn as nn
from sklearn import metrics
from time import strftime,localtime
from torch.utils.data import DataLoader
from transformers import BertModel,AdamW

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))

def setup_seed(seed):
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  random.seed(seed)
  torch.backends.cudnn.deterministic = True

fnametrainres = './dataset/Restaurants_corenlp/train.json'
fnametestres = './dataset/Restaurants_corenlp/test.json'
tokenizer = Tokenizer4BertGCN(85, 'bert-base-uncased')
trainset = globaltrainABSAGCNDATA(fnametrainres, tokenizer)
testset = globaltestABSAGCNDATA(fnametestres, tokenizer)

Training examples: 100%|██████████| 1119/1119 [01:31<00:00, 12.26it/s]


In [ ]:
class Instructor:
  def __init__(self,opt):
    self.opt = opt
    tokenizer = Tokenizer4BertGCN(85, 'bert-base-uncased')
    bert = BertModel.from_pretrained('bert-base-uncased')
    #model_class = LocalGCNBertClassifier(bert,2)
    self.model = globalGCNBertClassifier(bert,opt).to('cuda')
    #trainset = globalABSAGCNDATA(fnametrainres, tokenizer)
    #testset = globalABSAGCNDATAtest(fnametestres, tokenizer)
    self.train_dataloader = DataLoader(dataset=trainset, batch_size=16, shuffle=True)
    self.test_dataloader = DataLoader(dataset=testset, batch_size=16)
    
    if opt.device.type == 'cuda':
      logger.info('cuda memory allocated: {}'.format(torch.cuda.memory_allocated(self.opt.device.index)))
    self._print_args()

  def _print_args(self):
    n_trainable_params, n_nontrainable_params = 0, 0
    for p in self.model.parameters():
      n_params = torch.prod(torch.tensor(p.shape))
      if p.requires_grad:
        n_trainable_params += n_params
      else:
        n_nontrainable_params += n_params

    logger.info('n_trainable_params: {0}, n_nontrainable_params: {1}'.format(n_trainable_params, n_nontrainable_params))
    logger.info('training arguments:')
    
    for arg in vars(self.opt):
      logger.info('>>> {0}: {1}'.format(arg, getattr(self.opt, arg)))
  def _reset_params(self):
    for p in self.model.parameters():
      if p.requires_grad:
        if len(p.shape) > 1:
          #self.opt.initializer(p)   # xavier_uniform_
          torch.nn.init.xavier_uniform_(p)
        else:
          stdv = 1. / (p.shape[0]**0.5)
          torch.nn.init.uniform_(p, a=-stdv, b=stdv)
  def get_bert_optimizer(self,model):
    no_decay = ['bias','LayerNorm.weight']
    diff_part = ["bert.embeddings","bert.encoder"]
    logger.info("bert learning rate on")
    #quan zhong shuai jian: weight_decay,guo ni he 
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        'weight_decay': self.opt.weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(
            nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.opt.bert_lr, eps=self.opt.adam_epsilon)
    return optimizer

  def _train(self,criterion,optimizer,max_test_acc_overall=0):
    max_test_acc = 0
    max_f1 = 0
    global_step = 0
    model_path = ''
    for epoch in range(self.opt.num_epoch):
      logger.info('>' * 60)
      logger.info('epoch: {}'.format(epoch))
      n_correct, n_total = 0, 0
      for i_batch, sample_batched in enumerate(self.train_dataloader):
        global_step += 1
        # switch model to training mode, clear gradient accumulators
        self.model.train()
        optimizer.zero_grad()
        inputs = [sample_batched[col].to(self.opt.device) for col in self.opt.inputs_cols]
        outputs, penal = self.model(inputs)
        targets = sample_batched['polarity'].to(self.opt.device)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()
        
        if global_step % self.opt.log_step == 0:
          n_correct += (torch.argmax(outputs, -1) == targets).sum().item()
          n_total += len(outputs)
          train_acc = n_correct / n_total
          test_acc, f1 = self._evaluate()
          if test_acc > max_test_acc:
            max_test_acc = test_acc
            if test_acc > max_test_acc_overall:
              if not os.path.exists('./state_dict'):
                os.mkdir('./state_dict')
              model_path = './state_dict/{}_{}_acc_{:.4f}_f1_{:.4f}'.format("vigcn", "restaurants", test_acc, f1)
              self.best_model = copy.deepcopy(self.model)
              logger.info('>> saved: {}'.format(model_path))
          if f1 > max_f1:
            max_f1 = f1
          logger.info('loss: {:.4f}, acc: {:.4f}, test_acc: {:.4f}, f1: {:.4f}'.format(loss.item(), train_acc, test_acc, f1))
    return max_test_acc, max_f1, model_path
  def _evaluate(self, show_results=False):
    #switch model to evaluation mode
    self.model.eval()
    n_test_correct, n_test_total = 0, 0
    targets_all, outputs_all = None, None
    with torch.no_grad():
      for batch, sample_batched in enumerate(self.test_dataloader):
        inputs = [sample_batched[col].to(self.opt.device) for col in self.opt.inputs_cols]
        targets = sample_batched['polarity'].to(self.opt.device)
        outputs, penal = self.model(inputs)
        n_test_correct += (torch.argmax(outputs, -1) == targets).sum().item()
        n_test_total += len(outputs)
        targets_all = torch.cat((targets_all, targets), dim=0) if targets_all is not None else targets
        outputs_all = torch.cat((outputs_all, outputs), dim=0) if outputs_all is not None else outputs
    test_acc = n_test_correct / n_test_total
    f1 = metrics.f1_score(targets_all.cpu(), torch.argmax(outputs_all, -1).cpu(), labels=[0, 1, 2], average='macro')

    labels = targets_all.data.cpu()
    predic = torch.argmax(outputs_all, -1).cpu()
    if show_results:
      report = metrics.classification_report(labels, predic, digits=4)
      confusion = metrics.confusion_matrix(labels, predic)
      return report, confusion, test_acc, f1

    return test_acc, f1
  
  def _test(self):
    self.model = self.best_model
    self.model.eval()
    test_report, test_confusion, acc, f1 = self._evaluate(show_results=True)
    logger.info("Precision, Recall and F1-Score...")
    logger.info(test_report)
    logger.info("Confusion Matrix...")
    logger.info(test_confusion)
  
  def run(self):
    criterion = nn.CrossEntropyLoss()
    optimizer = self.get_bert_optimizer(self.model)
    max_test_acc_overall = 0
    max_f1_overall = 0
    #if 'bert' not in self.opt.model_name:
        #self._reset_params()
    max_test_acc, max_f1, model_path = self._train(criterion, optimizer, max_test_acc_overall)
    logger.info('max_test_acc: {0}, max_f1: {1}'.format(max_test_acc, max_f1))
    max_test_acc_overall = max(max_test_acc, max_test_acc_overall)
    max_f1_overall = max(max_f1, max_f1_overall)
    torch.save(self.best_model.state_dict(), model_path)
    logger.info('>> saved: {}'.format(model_path))
    logger.info('#' * 60)
    logger.info('max_test_acc_overall:{}'.format(max_test_acc_overall))
    logger.info('max_f1_overall:{}'.format(max_f1_overall))
    self._test()


In [ ]:
inputs_colses = ['text_bert_indices','text_raw_bert_indices','aspect_bert_indices','bert_segments_ids', 'attention_mask', 'asp_start', 'asp_end', 'adj_matrix', 'src_mask', 'aspect_mask']

In [ ]:
initializers = {
        'xavier_uniform_': torch.nn.init.xavier_uniform_,
        'xavier_normal_': torch.nn.init.xavier_normal_,
        'orthogonal_': torch.nn.init.orthogonal_,
}
optimizers = {
        'adadelta': torch.optim.Adadelta,
        'adagrad': torch.optim.Adagrad, 
        'adam': torch.optim.Adam,
        'adamax': torch.optim.Adamax, 
        'asgd': torch.optim.ASGD,
        'rmsprop': torch.optim.RMSprop,
        'sgd': torch.optim.SGD,
}
   

In [ ]:
import argparse
parser = argparse.ArgumentParser()
#parser.add_argument('--model_name', default='bert', type=str)
#parser.add_argument('--dataset', default='laptop', type=str, help=', '.join(dataset_files.keys()))
parser.add_argument('--optimizer', default='adam', type=str, help=', '.join(optimizers.keys()))
parser.add_argument('--initializer', default='xavier_uniform_', type=str, help=', '.join(initializers.keys()))
parser.add_argument('--learning_rate', default=0.002, type=float)
parser.add_argument('--l2reg', default=1e-4, type=float)
parser.add_argument('--num_epoch', default=15, type=int)
parser.add_argument('--batch_size', default=16, type=int)
parser.add_argument('--log_step', default=5, type=int)
parser.add_argument('--embed_dim', default=300, type=int)
parser.add_argument('--post_dim', type=int, default=30, help='Position embedding dimension.')
parser.add_argument('--pos_dim', type=int, default=30, help='Pos embedding dimension.')
parser.add_argument('--hidden_dim', type=int, default=384, help='GCN mem dim.')#50 #768
parser.add_argument('--num_layers', type=int, default=2, help='Num of GCN layers.')
parser.add_argument('--polarities_dim', default=3, type=int, help='3')

parser.add_argument('--input_dropout', type=float, default=0.7, help='Input dropout rate.')
parser.add_argument('--gcn_dropout', type=float, default=0.1, help='GCN layer dropout rate.')
parser.add_argument('--lower', default=True, help='Lowercase all words.')
parser.add_argument('--direct', default=False, help='directed graph or undirected graph')
parser.add_argument('--loop', default=True)

parser.add_argument('--bidirect', default=True, help='Do use bi-RNN layer.')
parser.add_argument('--rnn_hidden', type=int, default=50, help='RNN hidden state size.')
parser.add_argument('--rnn_layers', type=int, default=1, help='Number of RNN layers.')
parser.add_argument('--rnn_dropout', type=float, default=0.1, help='RNN dropout rate.')

parser.add_argument('--attention_heads', default=1, type=int, help='number of multi-attention heads')
parser.add_argument('--max_length', default=85, type=int)
parser.add_argument('--device', default=None, type=str, help='cpu, cuda')
parser.add_argument('--seed', default=1000, type=int) #0
parser.add_argument("--weight_decay", default=0.0, type=float, help="Weight decay if we apply some.")
parser.add_argument('--vocab_dir', type=str, default='./dataset/Restaurants_corenlp')
parser.add_argument('--pad_id', default=0, type=int)
parser.add_argument('--parseadj', default=False, action='store_true', help='dependency probability')
parser.add_argument('--parsehead', default=False, action='store_true', help='dependency tree')
parser.add_argument('--cuda', default='0', type=str)
parser.add_argument('--losstype', default=None, type=str, help="['doubleloss', 'orthogonalloss', 'differentiatedloss']")
parser.add_argument('--alpha', default=0.4, type=float) #0.25
parser.add_argument('--beta', default=0.3, type=float)  #0.25

# * bert
parser.add_argument('--pretrained_bert_name', default='bert-base-uncased', type=str)
parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
parser.add_argument('--bert_dim', type=int, default=768)
parser.add_argument('--bert_dropout', type=float, default=0.3, help='BERT dropout rate.')
parser.add_argument('--diff_lr', default=False, action='store_true')
parser.add_argument('--bert_lr', default=2e-5, type=float)
opt = parser.parse_args(args=[])

In [ ]:
opt.inputs_cols = inputs_colses
opt.initializer = initializers[opt.initializer]
opt.optimizer = optimizers[opt.optimizer]
opt.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') if opt.device is None else torch.device(opt.device)
setup_seed(opt.seed)

In [ ]:
if not os.path.exists('./log'):
        os.makedirs('./log', mode=0o777)
log_file = '{}-{}-{}.log'.format("vigcn",  "restaurants", strftime("%Y-%m-%d_%H:%M:%S", localtime()))
logger.addHandler(logging.FileHandler("%s/%s" % ('./log', log_file)))

In [ ]:
ins = Instructor(opt)
ins.run()